# Flights Table Builder

This is a copy of the flights_table_builder notebook that can be used to create route data for previous years.
This document currently uses a copy of route data from 1999 (621587766_T_T100_SEGMENT_ALL_CARRIER.csv), which was downloaded from the Bureau of Transportation Statistics and can be found in Google Drive. (See flights_table_builder.ipynb for more details on downloading BTS data.)

First, I'll import several libraries and then read in a list of airports from the Global Airport Database.

In [1]:
import pandas as pd
import numpy as np

data_year = '1999' # The year in which the flights in the BTS air travel dataset
# took place. This value will be added to the file name of the .csv file
# containing route data.
data_file = '621587766_T_T100_SEGMENT_ALL_CARRIER.csv'
# the BTS data to be imported into the program


airport_list = pd.read_csv('original_data\\GlobalAirportDatabase.txt',
delimiter=':',names=['icao_code', 'iata_code', 'airport_name', 'city/town', 
'country', 'lat_deg', 'lat_min', 'lat_sec', 'lat_dir', 'lon_deg', 'lon_min', 
'lon_sec', 'lon_dir', 'altitude', 'latitude_dec_deg', 'longitude_dec_deg']) 
# Headers are not included in the CSV, so they are instead based on those in the
# dictionary here:
# https://www.partow.net/miscellaneous/airportdatabase/index.html )

airport_list

,icao_code,iata_code,airport_name,city/town,country,lat_deg,lat_min,lat_sec,lat_dir,lon_deg,lon_min,lon_sec,lon_dir,altitude,latitude_dec_deg,longitude_dec_deg
0,AYGA,GKA,GOROKA,GOROKA,PAPUA NEW GUINEA,6,4,54,S,145,23,30,E,1610,-6.082,145.392
1,AYLA,LAE,NaN,LAE,PAPUA NEW GUINEA,0,0,0,U,0,0,0,U,0,0.000,0.000
2,AYMD,MAG,MADANG,MADANG,PAPUA NEW GUINEA,5,12,25,S,145,47,19,E,7,-5.207,145.789
3,AYMH,HGU,MOUNT HAGEN,MOUNT HAGEN,PAPUA NEW GUINEA,5,49,34,S,144,17,46,E,1643,-5.826,144.296
4,AYNZ,LAE,NADZAB,NADZAB,PAPUA NEW GUINEA,6,34,11,S,146,43,34,E,73,-6.570,146.726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9295,ZYTK,NaN,NaN,SHENYANG,CHINA,0,0,0,U,0,0,0,U,0,0.000,0.000
9296,ZYTL,DLC,ZHOUSHUIZI,DALIAN,CHINA,38,57,56,N,121,32,18,E,33,38.966,121.538
9297,ZYXC,NaN,NaN,XIANCHENG,CHINA,0,0,0,U,0,0,0,U,0,0.000,0.000
9298,ZYYC,NaN,NaN,YICHUN,CHINA,0,0,0,U,0,0,0,U,0,0.000,0.000


Next, I'll create two lists that feature each airport's IATA code, latitude, and longitude. These lists will then be merged into flight data from the Bureau of Transportation Statistics.

In [2]:
origin_airport_list_for_merge = airport_list[['iata_code', 'latitude_dec_deg', 
'longitude_dec_deg']].copy()
origin_airport_list_for_merge.rename(columns={'iata_code': 'origin_iata_code',
'latitude_dec_deg':'origin_lat','longitude_dec_deg':'origin_lon'},inplace=True)
origin_airport_list_for_merge

,origin_iata_code,origin_lat,origin_lon
0,GKA,-6.082,145.392
1,LAE,0.000,0.000
2,MAG,-5.207,145.789
3,HGU,-5.826,144.296
4,LAE,-6.570,146.726
...,...,...,...
9295,NaN,0.000,0.000
9296,DLC,38.966,121.538
9297,NaN,0.000,0.000
9298,NaN,0.000,0.000


In [3]:
destination_airport_list_for_merge = airport_list[['iata_code', 
'latitude_dec_deg', 'longitude_dec_deg']].copy()
destination_airport_list_for_merge.rename(columns={'iata_code': 
'destination_iata_code','latitude_dec_deg':'destination_lat',
'longitude_dec_deg':'destination_lon'},inplace=True)
destination_airport_list_for_merge

,destination_iata_code,destination_lat,destination_lon
0,GKA,-6.082,145.392
1,LAE,0.000,0.000
2,MAG,-5.207,145.789
3,HGU,-5.826,144.296
4,LAE,-6.570,146.726
...,...,...,...
9295,NaN,0.000,0.000
9296,DLC,38.966,121.538
9297,NaN,0.000,0.000
9298,NaN,0.000,0.000


Next, I'll import flight data from a .csv file that I downloaded from the Bureau of Transportation Statistics. BTS is a **great** resource for flight information because the data is entirely free. As its website states, "There are no copyright restrictions or charges for use on DOT publications." https://transportation.libanswers.com/faq/156950

The csv file below contains Air Carriers : T-100 Segment (All Carriers) data from all available months in 2018 (or another year of your choice). It can be found at https://www.transtats.bts.gov/DL_SelectFields.asp?gnoyr_VQ=FMG. I believe I downloaded all available fields (although you have the option to select only certain ones).

In [4]:
route_list = pd.read_csv(
    'original_data\\'+data_file, 
    low_memory = False) 
route_list

,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,PAYLOAD,SEATS,PASSENGERS,FREIGHT,MAIL,DISTANCE,RAMP_TO_RAMP,AIR_TIME,...,AIRCRAFT_GROUP,AIRCRAFT_TYPE,AIRCRAFT_CONFIG,YEAR,QUARTER,MONTH,DISTANCE_GROUP,CLASS,DATA_SOURCE,Unnamed: 50
0,0.0,4.0,200000.0,452.0,229.0,0.0,0.0,151.0,163.0,115.0,...,7,710,1,1999,2,4,1,L,DU,NaN
1,0.0,1.0,6000.0,0.0,0.0,428.0,0.0,481.0,91.0,80.0,...,6,681,2,1999,2,4,1,P,DU,NaN
2,0.0,1.0,34100.0,137.0,121.0,0.0,0.0,515.0,101.0,92.0,...,6,619,1,1999,2,4,2,F,DU,NaN
3,0.0,1.0,21500.0,0.0,0.0,0.0,0.0,152.0,43.0,25.0,...,6,635,2,1999,2,4,1,P,DU,NaN
4,0.0,1.0,21500.0,0.0,0.0,1914.0,0.0,454.0,90.0,75.0,...,6,635,2,1999,2,4,1,P,DU,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249446,960.0,855.0,21953520.0,101313.0,64574.0,98386.0,342519.0,100.0,26971.0,19535.0,...,6,620,1,1999,4,11,1,F,DU,NaN
249447,960.0,853.0,21908435.0,101114.0,62013.0,168911.0,50672.0,100.0,30988.0,19880.0,...,6,620,1,1999,4,11,1,F,DU,NaN
249448,992.0,882.0,22645080.0,104502.0,53696.0,108213.0,606572.0,100.0,28775.0,20698.0,...,6,620,1,1999,4,12,1,F,DU,NaN
249449,992.0,888.0,22797660.0,105204.0,57784.0,185088.0,148406.0,100.0,31991.0,20556.0,...,6,620,1,1999,4,12,1,F,DU,NaN


Here are the fields available within this list:

In [5]:
route_list.columns

Index(['DEPARTURES_SCHEDULED', 'DEPARTURES_PERFORMED', 'PAYLOAD', 'SEATS',
       'PASSENGERS', 'FREIGHT', 'MAIL', 'DISTANCE', 'RAMP_TO_RAMP', 'AIR_TIME',
       'UNIQUE_CARRIER', 'AIRLINE_ID', 'UNIQUE_CARRIER_NAME',
       'UNIQUE_CARRIER_ENTITY', 'REGION', 'CARRIER', 'CARRIER_NAME',
       'CARRIER_GROUP', 'CARRIER_GROUP_NEW', 'ORIGIN_AIRPORT_ID',
       'ORIGIN_AIRPORT_SEQ_ID', 'ORIGIN_CITY_MARKET_ID', 'ORIGIN',
       'ORIGIN_CITY_NAME', 'ORIGIN_STATE_ABR', 'ORIGIN_STATE_FIPS',
       'ORIGIN_STATE_NM', 'ORIGIN_COUNTRY', 'ORIGIN_COUNTRY_NAME',
       'ORIGIN_WAC', 'DEST_AIRPORT_ID', 'DEST_AIRPORT_SEQ_ID',
       'DEST_CITY_MARKET_ID', 'DEST', 'DEST_CITY_NAME', 'DEST_STATE_ABR',
       'DEST_STATE_FIPS', 'DEST_STATE_NM', 'DEST_COUNTRY', 'DEST_COUNTRY_NAME',
       'DEST_WAC', 'AIRCRAFT_GROUP', 'AIRCRAFT_TYPE', 'AIRCRAFT_CONFIG',
       'YEAR', 'QUARTER', 'MONTH', 'DISTANCE_GROUP', 'CLASS', 'DATA_SOURCE',
       'Unnamed: 50'],
      dtype='object')

The same page from which I downloaded the routes data also contains links to lookup tables that provide translations of aircraft codes. In the following cell, I import these codes into DataFrames that can then be merged with the routes table. 

In [6]:
aircraft_type = pd.read_csv('original_data\\L_AIRCRAFT_TYPE.csv')
aircraft_type.rename(columns={'Description':'Plane_Type_Text'},inplace=True)
aircraft_group = pd.read_csv('original_data\\L_AIRCRAFT_GROUP.csv')
aircraft_group.rename(columns={'Description':'Plane_Group_Text'},inplace=True)
aircraft_config = pd.read_csv('original_data\\L_AIRCRAFT_CONFIG.csv')
aircraft_config.rename(columns={'Description':'Plane_Config_Text'},inplace=True)
aircraft_type

,Code,Plane_Type_Text
0,7,Aero Commander 200
1,8,Aero Macchi AL-60
2,9,Aeronca 7-AC
3,10,Beech Bonanza 35A/C/D/E/G/H/J/K/S/V/ 36A
4,20,Bellanca CH-300
...,...,...
423,887,B787-800 Dreamliner
424,888,Boeing 737-900ER
425,889,B787-900 Dreamliner
426,890,Antonov 225 (6 Engine)


Next, I'll merge all of these tables together. Note that both the origin and destination airport lists are merged onto the routes table; this allows for the coordinates of both airport types to be plotted when creating route maps.

In [7]:
df_routes_planes_airports = route_list.merge(aircraft_type, how = 'left', 
left_on = 'AIRCRAFT_TYPE', right_on = 'Code')
df_routes_planes_airports = df_routes_planes_airports.merge(aircraft_group, 
how = 'left', left_on = 'AIRCRAFT_GROUP', right_on = 'Code')
df_routes_planes_airports = df_routes_planes_airports.merge(aircraft_config, 
how = 'left', left_on = 'AIRCRAFT_CONFIG', right_on = 'Code')
df_routes_planes_airports = df_routes_planes_airports.merge(
    origin_airport_list_for_merge, how = 'left', left_on = 'ORIGIN',
    right_on = 'origin_iata_code') 
df_routes_planes_airports = df_routes_planes_airports.merge(
    destination_airport_list_for_merge, how = 'left', left_on = 'DEST',
    right_on = 'destination_iata_code') 
df_routes_planes_airports

,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,PAYLOAD,SEATS,PASSENGERS,FREIGHT,MAIL,DISTANCE,RAMP_TO_RAMP,AIR_TIME,...,Code_y,Plane_Group_Text,Code,Plane_Config_Text,origin_iata_code,origin_lat,origin_lon,destination_iata_code,destination_lat,destination_lon
0,0.0,4.0,200000.0,452.0,229.0,0.0,0.0,151.0,163.0,115.0,...,7,"Jet, 3-Engine",1,Passenger Configuration,BNA,36.124,-86.678,NaN,NaN,NaN
1,0.0,1.0,6000.0,0.0,0.0,428.0,0.0,481.0,91.0,80.0,...,6,"Jet, 2-Engine",2,Freight Configuration,BNA,36.124,-86.678,SHV,32.446,-93.826
2,0.0,1.0,34100.0,137.0,121.0,0.0,0.0,515.0,101.0,92.0,...,6,"Jet, 2-Engine",1,Passenger Configuration,BNA,36.124,-86.678,TUL,36.198,-95.888
3,0.0,1.0,21500.0,0.0,0.0,0.0,0.0,152.0,43.0,25.0,...,6,"Jet, 2-Engine",2,Freight Configuration,BNA,36.124,-86.678,TYS,35.812,-83.993
4,0.0,1.0,21500.0,0.0,0.0,1914.0,0.0,454.0,90.0,75.0,...,6,"Jet, 2-Engine",2,Freight Configuration,BNA,36.124,-86.678,YIP,42.238,-83.530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253931,960.0,855.0,21953520.0,101313.0,64574.0,98386.0,342519.0,100.0,26971.0,19535.0,...,6,"Jet, 2-Engine",1,Passenger Configuration,OGG,20.899,-156.431,HNL,21.316,-157.927
253932,960.0,853.0,21908435.0,101114.0,62013.0,168911.0,50672.0,100.0,30988.0,19880.0,...,6,"Jet, 2-Engine",1,Passenger Configuration,HNL,21.316,-157.927,OGG,20.899,-156.431
253933,992.0,882.0,22645080.0,104502.0,53696.0,108213.0,606572.0,100.0,28775.0,20698.0,...,6,"Jet, 2-Engine",1,Passenger Configuration,OGG,20.899,-156.431,HNL,21.316,-157.927
253934,992.0,888.0,22797660.0,105204.0,57784.0,185088.0,148406.0,100.0,31991.0,20556.0,...,6,"Jet, 2-Engine",1,Passenger Configuration,HNL,21.316,-157.927,OGG,20.899,-156.431


This file will now be copied and modified to make it more suitable for route map creation. 


First, to prevent flights from 'Null Island' (https://en.wikipedia.org/wiki/Null_Island) from being displayed and to prevent other mapping errors, I'll remove rows with missing airport data and with coordinates of 0. (This assumes that no airport is exactly on the equator or on the prime meridian.)

In [8]:
df_routes_planes_airports_for_mapping = df_routes_planes_airports.copy()
df_routes_planes_airports_for_mapping.dropna(axis=0,subset=['origin_iata_code',
 'destination_iata_code', 'destination_lat', 'destination_lon'],inplace=True) 
df_routes_planes_airports_for_mapping = df_routes_planes_airports_for_mapping.query(
    "origin_lat != 0 and origin_lon != 0 and destination_lat != 0 \
        and destination_lon != 0")
df_routes_planes_airports_for_mapping.reset_index(
    drop=True,inplace=True)
    

The following cell creates a new column showing the origin and destination airports separated by an underscore. These airports are sorted alphabetically, which allows flights from Airport B to Airport A and from Airport A to Airport B to be consolidated together within the route mapping notebook. Using np.where() as opposed to a for loop saved a considerable amount of time.



In [9]:
df_routes_planes_airports_for_mapping['ORIGIN_DEST'] = \
np.where(df_routes_planes_airports_for_mapping['ORIGIN'] < \
df_routes_planes_airports_for_mapping['DEST'], 
df_routes_planes_airports_for_mapping['ORIGIN'] + '_' + \
df_routes_planes_airports_for_mapping['DEST'], 
df_routes_planes_airports_for_mapping['DEST'] + \
'_' + df_routes_planes_airports_for_mapping['ORIGIN'])

# The following for loop took over 23 minutes to run, 
# whereas the np.where() approach took 0.2 seconds!
# df_routes_planes_airports_for_mapping_v2.reset_index(drop=True,inplace=True)
# df_routes_planes_airports_for_mapping_v2['ORIGIN_DEST'] = ''
# loop_start_time = time.time()
# for i in range(len(df_routes_planes_airports_for_mapping_v2)):
#     if i % 1000 == 0:
#         print("Now on row",i)
#         print("elapsed time (in seconds):", time.time() - loop_start_time)
#     origin_airport = \
#     df_routes_planes_airports_for_mapping_v2.iloc[i, 
#     df_routes_planes_airports_for_mapping_v2.columns.get_loc('ORIGIN')]
#     destination_airport = \
#     df_routes_planes_airports_for_mapping_v2.iloc[i, 
#     df_routes_planes_airports_for_mapping_v2.columns.get_loc('DEST')]
#     if origin_airport < destination_airport:
#         origin_dest = origin_airport+'_'+destination_airport
#     else:
#         origin_dest = destination_airport+'_'+origin_airport
#     df_routes_planes_airports_for_mapping_v2.iloc[i, 
#     df_routes_planes_airports_for_mapping_v2.columns.get_loc('ORIGIN_DEST')] = \
#             origin_dest


In [10]:
df_routes_planes_airports_for_mapping

,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,PAYLOAD,SEATS,PASSENGERS,FREIGHT,MAIL,DISTANCE,RAMP_TO_RAMP,AIR_TIME,...,Plane_Group_Text,Code,Plane_Config_Text,origin_iata_code,origin_lat,origin_lon,destination_iata_code,destination_lat,destination_lon,ORIGIN_DEST
0,0.0,1.0,6000.0,0.0,0.0,428.0,0.0,481.0,91.0,80.0,...,"Jet, 2-Engine",2,Freight Configuration,BNA,36.124,-86.678,SHV,32.446,-93.826,BNA_SHV
1,0.0,1.0,34100.0,137.0,121.0,0.0,0.0,515.0,101.0,92.0,...,"Jet, 2-Engine",1,Passenger Configuration,BNA,36.124,-86.678,TUL,36.198,-95.888,BNA_TUL
2,0.0,1.0,21500.0,0.0,0.0,0.0,0.0,152.0,43.0,25.0,...,"Jet, 2-Engine",2,Freight Configuration,BNA,36.124,-86.678,TYS,35.812,-83.993,BNA_TYS
3,0.0,1.0,21500.0,0.0,0.0,1914.0,0.0,454.0,90.0,75.0,...,"Jet, 2-Engine",2,Freight Configuration,BNA,36.124,-86.678,YIP,42.238,-83.530,BNA_YIP
4,0.0,29.0,648896.0,2572.0,1785.0,122.0,418.0,649.0,3341.0,2904.0,...,"Jet, 4-Engine/6-Engine",1,Passenger Configuration,BOI,43.564,-116.223,DEN,39.858,-104.667,BOI_DEN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217537,960.0,855.0,21953520.0,101313.0,64574.0,98386.0,342519.0,100.0,26971.0,19535.0,...,"Jet, 2-Engine",1,Passenger Configuration,OGG,20.899,-156.431,HNL,21.316,-157.927,HNL_OGG
217538,960.0,853.0,21908435.0,101114.0,62013.0,168911.0,50672.0,100.0,30988.0,19880.0,...,"Jet, 2-Engine",1,Passenger Configuration,HNL,21.316,-157.927,OGG,20.899,-156.431,HNL_OGG
217539,992.0,882.0,22645080.0,104502.0,53696.0,108213.0,606572.0,100.0,28775.0,20698.0,...,"Jet, 2-Engine",1,Passenger Configuration,OGG,20.899,-156.431,HNL,21.316,-157.927,HNL_OGG
217540,992.0,888.0,22797660.0,105204.0,57784.0,185088.0,148406.0,100.0,31991.0,20556.0,...,"Jet, 2-Engine",1,Passenger Configuration,HNL,21.316,-157.927,OGG,20.899,-156.431,HNL_OGG


Finally, I'll save this file as a .csv so that it can be imported into route_maps_builder_v2.ipynb.

In [11]:
df_routes_planes_airports_for_mapping.to_csv(
    'routes_planes_coordinates_for_mapping_'+data_year+'.csv')